### <font color = '5tan'>Code creator name: Tan Chuan Kuan </font>


### <font color = 'navy'>Automate collecting rental flats information on HDB rental website</font>
### <font color = 'green'>Code Create Date: Sep-2021 </font>

#### <font color = 'blue'>This code helps to automate in collecting several tables of HDB rental information.
#### <font color = 'blue'>This comes handy when someone wishes to get rental information from multiple blocks of flats in the same street.</font>

In [2]:
import pandas as pd
from selenium import webdriver
# https://chromedriver.chromium.org/downloads

from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select # for selecting dropdown list
from selenium.webdriver.common.
import time

In [47]:
#initializing chrome by making an object of it
# driver=webdriver.Chrome(ChromeDriverManager().ipynb_checkpoints/install()) 
driver = webdriver.Chrome(r'chromedriver.exe')
driver.maximize_window() #Maximize the window screen
url = "https://services2.hdb.gov.sg/webapp/BR12AWRentalEnq/BR12PSearch.jsp"
driver.get(url) 
time.sleep(1) #Wait for 1sec

############################################
blk_lists = ['79a', '79b', '79c', '79d', '79e']
street_name = 'Toa Payoh Central'
############################################

columnNames = ['Rental commence mth',
                'Town',
                'Blk no',
                'Street Name',
                'Flat Type',
                'Mnthly Rent (S$)']
# table = pd.DataFrame(None, columns= columnNames)
table = list() # empty list collect rows of info

# month dict for selecing active calendar button
month = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

for blk_list in blk_lists:
    
    # click the 'Block / Street' radio button
    driver.find_element(By.XPATH,
                '//*[@id="panelContent"]/div/div[3]/fieldset/div[2]/label[2]').click()
    driver.implicitly_wait(1) #Wait for 1sec
    
    # key blk number into the textbox
    driver.find_element(By.XPATH,'//*[@id="txtBlk"]').send_keys(blk_list) 
    driver.implicitly_wait(1) #Wait for 1sec
    
    # click the 'Get Street' button
    driver.find_element(By.XPATH,'//*[@id="btnRetrieveStreet"]').click() 
    driver.implicitly_wait(1) #Wait for 1sec
    
    # Select street name from dropdown list
    select = Select(driver.find_element(By.XPATH,'//*[@id="selStreet"]')) 
    select.select_by_visible_text(street_name)
    
    # click the 'from' calendar button
    driver.find_element(By.XPATH,
                '//*[@id="panelSearchByBlk"]/div[4]/div[2]/div[1]/div[1]/div/div[2]/a').click()
    driver.implicitly_wait(1) #Wait for 1sec

    # click '<' arrow to year 2020
    driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/table/thead/tr/th[1]/i').click() 
    driver.implicitly_wait(1) #Wait for 1sec
    
    
    #Search for a clickable calendar button in monthly order
    for num in range(1,12):
        try: 
            #############################################################################
            # very slow in try looping to another month (button)... 1 sec+ per button loop
            driver.find_element(By.XPATH,f'/html/body/div[3]/div[2]/table/tbody/tr/td/span[{num}]').click()
            # if able to click, end the for loop, cos it has found a clickable month button
            break
        except:
            pass
            # print(f"{month.get(num)} button disabled in calender!!")

    driver.implicitly_wait(1) #Wait for 1sec
        
    # Below 4 lines not needed as website has, by default, set the latest month for the 'to' month      
    
    # click the 'to' calendar button
#    driver.find_element(By.XPATH,
#                '//*[@id="panelSearchByBlk"]/div[4]/div[2]/div[1]/div[3]/div/div[2]/a').click()
#    driver.implicitly_wait(1) #Wait for 1sec
    
    #click September-2021
#     driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/table/tbody/tr/td/span[9]').click() 
#     driver.implicitly_wait(1) #Wait for 1sec
    
    # click 'I Accept' button
    driver.find_element(By.XPATH,'//*[@id="btnSearch"]').click() 
    driver.implicitly_wait(1) #Wait for 1sec

    # Xpath for the entire table
    row_nums = 0 # Initialise. prepare for no result for the blk. If not, next for loop will give error
    try:
        tableData = driver.find_element_by_xpath('//*[@id="result-paginated"]/div/table') 
        # Get the number of rows of data becos no. of rows will vary
        row_nums = len(tableData.text.splitlines())-1 # exclude the column row
    except:
        print(f"No result at {street_name} block {blk_list}")
    
    # Note: pd.read_html(driver.page_source) good only for single table. 
    # This case there is need to append tables
    for row_num in range(1,row_nums+1):
        
        row_list = [] # create an empty list for every loop
        for col_num in range(1,len(columnNames)+1): 
            # There are 6 columns: 
            # Rental commence mth, town, blk no, Street name, Flat type, monthly rent
            get_indiv_data = driver.find_element_by_xpath(
                f'//*[@id="result-paginated"]/div/table/tbody/tr[{row_num}]/td[{col_num}]')
            row_list.append(get_indiv_data.text) # forming a row of rental info
            
        table.append(row_list)
    

    # See if there is 2nd page, meaning more than 10 units rent out for the block
    # code assumes there are max of 2 pages as unlikely to have more than 20 units per block are rented out
    try:
        # try get Xpath for "page 2" button, if there is one.
        driver.find_element(By.XPATH, '//*[@id="page1"]').click()
        driver.implicitly_wait(1) #Wait for 1sec
     
        # Xpath for the entire table
        tableData = driver.find_element_by_xpath('//*[@id="result-paginated"]/div/table') 
        # Get the number of rows of data becos no. of rows will vary
        row_nums = len(tableData.text.splitlines()) -1 # exclude the column row
         
        # Note: pd.read_html(driver.page_source) good only for single table. 
        # This case there is need to append tables
        for row_num in range(11,row_nums+11):
            row_list = [] # create an empty list for every loop
            for col_num in range(1,len(columnNames)+1): 
                # There are 6 columns: 
                # Rental commence mth, town, blk no, Street name, Flat type, monthly rent
                get_indiv_data = driver.find_element_by_xpath(
                     f'//*[@id="result-paginated"]/div/table/tbody/tr[{row_num}]/td[{col_num}]')
                row_list.append(get_indiv_data.text) # forming a row of rental info
            table.append(row_list)
        
        print(street_name,'block', blk_list, 'has more than 1 page with',row_nums+10,'units rented out.')
    
    except:
        print(street_name,'block', blk_list, 'has 1 page with',row_nums,'units rented out.')
    
    finally:
        # if it is the last block, Do Not click 'New Enquiry' button and exit the for loop
        if blk_list != blk_lists[-1]:
            # click 'New Enquiry' button
            driver.find_element(By.XPATH,'//*[@id="btnNewEnquiry"]').click() 
            driver.implicitly_wait(1) #Wait for 1sec 
        else:
            exit # or driver.quit() ??

driver.close() # close the driver instance, closing every associated window
        
# Insert the column names into dataframe
df = pd.DataFrame(table, columns= columnNames)
df

Toa Payoh Central block 79a has more than 1 page with 16 units rented out.
Toa Payoh Central block 79b has more than 1 page with 11 units rented out.
Toa Payoh Central block 79c has 1 page with 5 units rented out.
Toa Payoh Central block 79d has 1 page with 6 units rented out.
Toa Payoh Central block 79e has more than 1 page with 17 units rented out.


,Rental commence mth,Town,Blk no,Street Name,Flat Type,Mnthly Rent (S$)
0,Oct 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2200.00
1,Aug 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2400.00
2,Aug 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2700.00
3,Jul 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2550.00
4,Jul 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2600.00
5,Jul 2021,Toa Payoh,79A,Toa Payoh Central,5-Rm,2800.00
6,Jul 2021,Toa Payoh,79A,Toa Payoh Central,5-Rm,2900.00
7,Jul 2021,Toa Payoh,79A,Toa Payoh Central,5-Rm,2900.00
8,Jun 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2450.00
9,May 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2200.00
